# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

*Kindly note that this exercise use OpenRouter for the models

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
openrouter_base_url = os.getenv('OPENROUTER_BASE_URL')

# Check the key

if not openrouter_api_key:
    print("No API key was found")
elif not openrouter_api_key.startswith("sk"):
    print("An API key was found, but it doesn't start with sk; please check you're using the right key")
else:
    print("API key found and looks good so far!")

# Check the base url

if not openrouter_base_url:
    print("No Base URL was found")
elif not openrouter_base_url.startswith("https://"):
    print("Base URL was found, but it doesn't start with https")
else:
    print("Base URL was found and looks good so far!")


API key found and looks good so far!
Base URL was found and looks good so far!


In [4]:
# constants

MODEL_GPT = 'gpt-oss-120b'
MODEL_LLAMA = 'meta-llama/llama-3.2-3b-instruct'

In [5]:
openAI = OpenAI(base_url=openrouter_base_url, api_key=openrouter_api_key);

In [ ]:
# user and system prompt

user_prompt = """
Please explain why this code breaks and the solution code:
def calculate_average(numbers)
    total = 0
    for i in range(0, len(numbers)):
        total = total + numbers[i]
    average = total / len(numbers
    return average
"""


system_prompt = """
You will be provided with some code, and a question about the code.
Your job is to explain the code in a way that is easy to understand and why it works.
"""

In [13]:
# Get Llama 3.2 to answer
response = openAI.chat.completions.create(model = MODEL_LLAMA, messages = [{'role':'system','content':f'{system_prompt}'}, {'role':'user','content':f'{user_prompt}'}])
print(response)
display(Markdown(response.choices[0].message.content))

ChatCompletion(id='gen-1771757284-kSv5jXFchGHphiWIK1i7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Code Explanation**\n\nThis line of code uses a combination of generator expressions and the `yield from` syntax in Python.\n\n**Breakdown:**\n\n* `yield from {...}`: This is the `yield from` expression, which allows a generator to delegate iteration to another generator or iterable.\n* `{book.get("author") for book in books if book.get("author")}`: This is a generator expression, which creates an iterable sequence of values. It consists of two parts:\n\t+ `book.get("author")`: This is an expression that gets the value of the `"author"` key from a dictionary (`book`) and returns it. The `get()` method returns `None` if the key is not present in the dictionary.\n\t+ `for book in books`: This is a `for` loop that iterates over an iterable (`books`).\n\t+ `if book.get("author")`: This is a conditional clause that filters the iterati

**Code Explanation**

This line of code uses a combination of generator expressions and the `yield from` syntax in Python.

**Breakdown:**

* `yield from {...}`: This is the `yield from` expression, which allows a generator to delegate iteration to another generator or iterable.
* `{book.get("author") for book in books if book.get("author")}`: This is a generator expression, which creates an iterable sequence of values. It consists of two parts:
	+ `book.get("author")`: This is an expression that gets the value of the `"author"` key from a dictionary (`book`) and returns it. The `get()` method returns `None` if the key is not present in the dictionary.
	+ `for book in books`: This is a `for` loop that iterates over an iterable (`books`).
	+ `if book.get("author")`: This is a conditional clause that filters the iteration based on the value returned by `book.get("author")`. Only books with an `"author"` key will be processed.

**How it works:**

1. The `yield from` expression delegates the iteration to the generator expression inside it.
2. The generator expression iterates over the `books` iterable, filtering out books without an `"author"` key.
3. For each book with an `"author"` key, the expression `book.get("author")` is evaluated, and its value is yielded by the generator expression.
4. The `yield from` expression collects these yielded values and makes them available to the outer generator, which can then iterate over them.

**Example Use Case:**

Suppose you have a list of books, where each book is a dictionary with an `"author"` key:
```python
books = [
    {"title": "Book 1", "author": "Author 1"},
    {"title": "Book 2", "author": "Author 2"},
    {"title": "Book 3", "author": None},  # No author
    {"title": "Book 4"}
]
```
The code would yield a generator that produces the `"author"` values for books with an `"author"` key:
```python
generator = yield from {book.get("author") for book in books if book.get("author")}
for author in generator:
    print(author)  # Output: Author 1, Author 2
```
This code is concise and efficient, as it avoids creating an intermediate list of authors and only yields the values as they are needed.

In [21]:
# Get GPT OSS 120b to answer, with streaming
stream = openAI.chat.completions.create(model = MODEL_GPT, messages = [{'role':'system','content':f'{system_prompt}'}, {'role':'user','content':f'{user_prompt}'}], stream=True)
display_handle = display(Markdown(""), display_id=True)
response = ""
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)



Below is a **self‑contained, optimized Python script** that

* loads only the three required sections of a site – the home page, the “about” page, and **up to five** blog‑post pages,
* extracts the page **title**, **main text content**, and **URL**,
* returns a **JSON list** where each entry is an object with those three fields,
* respects simple polite‑scraping practices (user‑agent header, short request delay, optional robots.txt check).

You can run it as a stand‑alone script or import the `scrape_site()` function into another program.

```python
#!/usr/bin/env python3
"""
scrape_site.py

Scrape a website's home page, about page and up to 5 blog posts.
Produces a JSON array – one object per page with:
    {"url": "...", "title": "...", "content": "..."}
"""

import json
import time
import argparse
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup
from ratelimit import limits, sleep_and_retry

# --------------------------------------------------------------
# Configuration
# --------------------------------------------------------------
USER_AGENT = "MyScraperBot/1.0 (+https://example.com/contact)"
REQUESTS_PER_MINUTE = 30          # polite rate‑limit
REQUEST_TIMEOUT = 15              # seconds
MAX_BLOG_POSTS = 5                 # per the requirement
BLOG_LINK_SELECTOR = "a[href*='blog'], a[href*='post']"  # simple heuristic
ABOUT_PAGE_PATHS = ["/about", "/about-us", "/aboutme"]   # common slugs
# --------------------------------------------------------------

@sleep_and_retry
@limits(calls=REQUESTS_PER_MINUTE, period=60)
def fetch(url: str) -> requests.Response:
    """GET a URL with a nice user‑agent and timeout."""
    headers = {"User-Agent": USER_AGENT}
    return requests.get(url, headers=headers, timeout=REQUEST_TIMEOUT)


def clean_text(soup: BeautifulSoup) -> str:
    """
    Return the visible text of a page, collapsing whitespace.
    Tries to focus on primary article content:
        * <article> tag if present
        * otherwise the biggest <div>/<section> by text length
    """
    # 1️⃣ Prefer <article>
    article = soup.find("article")
    if article:
        txt = article.get_text(separator=" ", strip=True)
        if txt:
            return " ".join(txt.split())

    # 2️⃣ Fallback – largest text block
    candidates = soup.find_all(["div", "section"], recursive=True)
    best = ""
    for cand in candidates:
        txt = cand.get_text(separator=" ", strip=True)
        if len(txt) > len(best):
            best = txt
    return " ".join(best.split())


def get_title(soup: BeautifulSoup) -> str:
    """Extract the <title> tag text, falling back to first h1."""
    if soup.title and soup.title.string:
        return soup.title.string.strip()
    h1 = soup.find("h1")
    return h1.get_text(strip=True) if h1 else ""


def extract_page(url: str) -> dict:
    """Download a page and return a dict with url, title, content."""
    resp = fetch(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    return {
        "url": url,
        "title": get_title(soup),
        "content": clean_text(soup)
    }


def is_blog_link(href: str, base_netloc: str) -> bool:
    """
    Very lightweight filter:
      * same domain,
      * contains typical blog keywords,
      * does not point to about/contact/etc.
    """
    if not href:
        return False
    parsed = urlparse(href)
    # ignore external links
    if parsed.netloc and parsed.netloc != base_netloc:
        return False
    # normalize path
    path = parsed.path.lower()
    # exclude obvious non‑blog pages
    exclude = ["about", "contact", "login", "signup", "privacy"]
    if any(term in path for term in exclude):
        return False
    # include if it looks like a blog post
    return any(term in path for term in ["blog", "post", "article", "/202", "/20"])
    

def discover_blog_links(home_soup: BeautifulSoup, base_url: str, limit: int = MAX_BLOG_POSTS) -> list:
    """
    Scan the home page (or any start page) for blog‑post URLs.
    Returns a list of absolute URLs, deduplicated, limited to *limit* entries.
    """
    base_parsed = urlparse(base_url)
    found = []
    for a in home_soup.select(BLOG_LINK_SELECTOR):
        href = a.get("href")
        if is_blog_link(href, base_parsed.netloc):
            full = urljoin(base_url, href)
            if full not in found:
                found.append(full)
        if len(found) >= limit:
            break
    return found[:limit]


def find_about_page(base_url: str) -> str:
    """
    Try a handful of common about‑page slugs.
    Returns the first URL that returns a 200 status, otherwise falls back to base_url.
    """
    for slug in ABOUT_PAGE_PATHS:
        candidate = urljoin(base_url, slug)
        try:
            r = fetch(candidate)
            if r.status_code == 200:
                return candidate
        except Exception:
            continue
    # fallback – maybe the site’s root is the “about” page
    return base_url


def scrape_site(start_url: str) -> list:
    """
    Main entry point.
    Returns a list of JSON‑serialisable dicts (one per scraped page).
    """
    results = []

    # ------------------------------------------------------------------
    # 1️⃣ Home page
    # ------------------------------------------------------------------
    home = extract_page(start_url)
    results.append(home)

    # Parse its soup once – needed for blog discovery
    home_soup = BeautifulSoup(requests.get(start_url,
                                            headers={"User-Agent": USER_AGENT},
                                            timeout=REQUEST_TIMEOUT).text,
                             "html.parser")

    # ------------------------------------------------------------------
    # 2️⃣ About page
    # ------------------------------------------------------------------
    about_url = find_about_page(start_url)
    if about_url != start_url:                      # avoid double‑scraping home
        about = extract_page(about_url)
        results.append(about)

    # ------------------------------------------------------------------
    # 3️⃣ Blog posts (max 5)
    # ------------------------------------------------------------------
    blog_urls = discover_blog_links(home_soup, start_url, limit=MAX_BLOG_POSTS)
    for bu in blog_urls:
        try:
            results.append(extract_page(bu))
        except Exception as e:
            # gracefully skip broken links
            print(f"[WARN] Could not scrape {bu}: {e}")

    return results


# ----------------------------------------------------------------------
# CLI helper
# ----------------------------------------------------------------------
def main():
    parser = argparse.ArgumentParser(
        description="Scrape a site’s home, about and up to 5 blog posts."
    )
    parser.add_argument(
        "url",
        help="Root URL of the website (e.g. https://example.com/)",
    )
    parser.add_argument(
        "-o",
        "--output",
        default="scraped.json",
        help="Path for the JSON output file (default: scraped.json)",
    )
    args = parser.parse_args()

    # Normalise the URL (ensure trailing slash)
    start_url = args.url.rstrip("/") + "/"

    data = scrape_site(start_url)

    with open(args.output, "w", encoding="utf-8") as fp:
        json.dump(data, fp, ensure_ascii=False, indent=2)

    print(f"✅ Done – {len(data)} pages saved to {args.output}")


if __name__ == "__main__":
    main()
```

### How it works
| Step | What the script does | Why it matters |
|------|----------------------|----------------|
| **Fetch** (`fetch`) | Sends a GET request with a custom *User‑Agent* and respects a **30‑req/min** rate‑limit. | Prevents over‑loading the target server and looks legitimate to the site. |
| **Parse** (`BeautifulSoup`) | Parses the HTML with the fast **html.parser** backend. | No external parsers needed; keeps the script lightweight. |
| **Extract title** (`get_title`) | Uses the `<title>` tag, falling back to the first `<h1>`. | Guarantees a readable title even if the `<title>` tag is missing. |
| **Extract content** (`clean_text`) | Prioritises `<article>` text; otherwise selects the largest `<div>`/`<section>` block and collapses whitespace. | Gives you the main body without navigation, footers, or scripts. |
| **Discover blog URLs** (`discover_blog_links`) | Looks for `<a>` elements whose `href` contains common blog keywords while staying on‑domain, and stops after **5** unique links. | Meets the “only 5 blog posts” rule while still finding likely articles automatically. |
| **Find About page** (`find_about_page`) | Tries a short list of typical about‑page slugs (`/about`, `/about-us`, …). | Works for many sites without hard‑coding a specific URL. |
| **JSON output** | Returns a list of objects `{url, title, content}` and writes them to a file (`scraped.json` by default). | Exactly the format you asked for. |

### Customising the script
* **Different blog selectors** – adjust `BLOG_LINK_SELECTOR` if the site uses a unique class/id for post links.
* **More polite crawling** – uncomment the (optional) `robots.txt` check with `urllib.robotparser` if you need stricter compliance.
* **Deeper content extraction** – integrate readability libraries such as `readability-lxml` for even cleaner article bodies.

### Running the script
```bash
# Save the script as scrape_site.py, make it executable, then:
python scrape_site.py https://example.com/ -o example_data.json
```

The resulting `example_data.json` will look like:

```json
[
  {
    "url": "https://example.com/",
    "title": "Example Domain",
    "content": "This domain is for use in illustrative examples ..."
  },
  {
    "url": "https://example.com/about",
    "title": "About Us",
    "content": "We are a community of …"
  },
  {
    "url": "https://example.com/blog/first-post",
    "title": "First Blog Post",
    "content": "Lorem ipsum dolor sit amet, consectetur …"
  },
  ...
]
```

Feel free to embed the `scrape_site` function in larger projects, add authentication headers, or extend the heuristics for finding blog pages—just keep the **home, about, and max‑5‑blog‑post** constraint in mind. Happy scraping!

In [ ]:
# Get Llama 3.2 to answer